<a href="https://colab.research.google.com/github/dfukagaw28/ColabNotebooks/blob/main/%E9%9D%92%E7%A9%BA%E6%96%87%E5%BA%AB%E3%81%AE%E5%89%8D%E5%87%A6%E7%90%86%E6%B8%88%E3%81%BF%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E5%88%A9%E7%94%A8%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 青空文庫の前処理済みテキストデータを利用する

青空文庫のテキストデータををテキストマイニングや言語モデルの学習等の目的で利用する際には，ルビや注記などのノイズとなるものを除去するための前処理が必要です。

前処理済みの青空文庫テキストデータが Hugging Face 上で公開されています（ちょうど２年前頃。もう少し早く知りたかった）。

*   https://huggingface.co/datasets/globis-university/aozorabunko-clean
*   [青空文庫のテキストから作成したコーパスを Hugging Face で公開しました #LLM - Qiita](https://qiita.com/akeyhero/items/b53eae1c0bc4d54e321f)

[Hugging Face](https://huggingface.co/)（ハギングフェイス）とは、自然言語処理（NLP）や機械学習（ML）分野で広く利用されているオープンソースプラットフォーム兼コミュニティです。

音声・画像・テキストなどのデータセットや，自然言語モデル BERT やその変種・改良版などが Hugging Face 上で公開されており，誰でも利用することができます。
また，新たなデータセットやモデルを誰でも公開することができます。

Python で Hugging Face 状のデータセットを利用するには，Hugging Face が公開している Python ライブラリ [datasets](https://pypi.org/project/datasets/) を利用するのが便利です。Google Colab のランタイムには，この datasets が最初からインストールされており，追加インストール不要で import することができます（2025年10月時点）。

In [ ]:
%%time
#@title データセットを読み込む
from datasets import load_dataset
ds = load_dataset('globis-university/aozorabunko-clean')

In [2]:
# データセット
ds

DatasetDict({
    train: Dataset({
        features: ['text', 'footnote', 'meta'],
        num_rows: 16951
    })
})

In [3]:
# 訓練用データ
ds['train']

Dataset({
    features: ['text', 'footnote', 'meta'],
    num_rows: 16951
})

In [4]:
# 1つの作品を抽出する
book = ds['train'][0]
book['meta']['作品名']

'ウェストミンスター寺院'

In [5]:
# 1つの作品のテキストデータを抽出する
text = book['text']
print(f'テキストの文字数: {len(text)}')
print()
print('テキストの先頭100文字:')
print(text[:100])

テキストの文字数: 10639

テキストの先頭100文字:
深いおどろきにうたれて、
名高いウェストミンスターに
真鍮や石の記念碑となって
すべての王侯貴族が集まっているのをみれば、
今はさげすみも、ほこりも、見栄もない。
善にかえった貴人の姿、
華美と俗世の


In [6]:
# 1つの作品のフッター情報（主にテキスト入力作業に関する情報）を抽出する
print(book['footnote'])

底本：「スケッチ・ブック」新潮文庫、新潮社
　　　1957（昭和32）年5月20日発行
　　　2000（平成12）年2月20日33刷改版
※「寂莫」と「寂寞」の混在は、底本通りです。
入力：えにしだ
校正：砂場清隆
2020年3月28日作成
青空文庫作成ファイル：
このファイルは、インターネットの図書館、青空文庫（https://www.aozora.gr.jp/）で作られました。入力、校正、制作にあたったのは、ボランティアの皆さんです。



In [7]:
%%time
#@title 作品の文字種別を抽出する

import pandas as pd

series = pd.Series([row['meta']['文字遣い種別'] for row in ds['train']])

CPU times: user 6.03 s, sys: 61.7 ms, total: 6.09 s
Wall time: 6.14 s


In [8]:
# 文字遣い種別を集計する

series.value_counts()

,count
新字新仮名,10246
新字旧仮名,4515
旧字旧仮名,2143
旧字新仮名,32
その他,15


In [ ]:
#@title 作品名で絞り込む

# 作品名が「吾輩」で始まる作品を抽出する
temp = ds.filter(lambda row: row['meta']['作品名'].startswith('吾輩'))

print('ヒットした件数:', temp['train'].num_rows)

book = temp['train'][0]

In [10]:
book['meta']

{'作品ID': '000789',
 '作品名': '吾輩は猫である',
 '作品名読み': 'わがはいはねこである',
 'ソート用読み': 'わかはいはねこてある',
 '副題': '',
 '副題読み': '',
 '原題': '',
 '初出': '「ホトトギス」1905（明治38）年1月、2月、4月、6月、7月、10月、1906（明治39）年1月、3月、4月、8月',
 '分類番号': 'NDC 913',
 '文字遣い種別': '新字新仮名',
 '作品著作権フラグ': 'なし',
 '公開日': datetime.datetime(1999, 9, 21, 0, 0),
 '最終更新日': datetime.datetime(2018, 2, 5, 0, 0),
 '図書カードURL': 'https://www.aozora.gr.jp/cards/000148/card789.html',
 '人物ID': '000148',
 '姓': '夏目',
 '名': '漱石',
 '姓読み': 'なつめ',
 '名読み': 'そうせき',
 '姓読みソート用': 'なつめ',
 '名読みソート用': 'そうせき',
 '姓ローマ字': 'Natsume',
 '名ローマ字': 'Soseki',
 '役割フラグ': '著者',
 '生年月日': '1867-02-09',
 '没年月日': '1916-12-09',
 '人物著作権フラグ': 'なし',
 '底本名1': '夏目漱石全集1',
 '底本出版社名1': 'ちくま文庫、筑摩書房',
 '底本初版発行年1': '1987（昭和62）年9月29日',
 '入力に使用した版1': '1987（昭和62）年9月29日',
 '校正に使用した版1': '1994（平成6）年9月30日第4刷',
 '底本の親本名1': '筑摩全集類聚版\u3000夏目漱石全集\u30001',
 '底本の親本出版社名1': '筑摩書房',
 '底本の親本初版発行年1': '1971（昭和46）年4月5日',
 '底本名2': '',
 '底本出版社名2': '',
 '底本初版発行年2': '',
 '入力に使用した版2': '',
 '校正に使用した版2': '',
 '底本の親本名2': '',
 '底本の親本

## aozorahack/aozorabunko_text と比較する

青空文庫公式プロジェクト「Code for 青空文庫」に由来するテキストデータセットである aozorahack/aozorabunko_text と比較する。

*   Hugging Face の globis-university/aozorabunko-clean データセットについて
    *   最終更新日時は 2023年7月頃
    *   作品数は 16951
*   aozorahack/aozorabunko_text データセットについて
    *   最終更新日時は 2023年3月頃
    *   作品数は 17436

In [11]:
!git clone --depth=1 https://github.com/aozorahack/aozorabunko_text.git
!find aozorabunko_text/cards -type f | wc -l

Cloning into 'aozorabunko_text'...
remote: Enumerating objects: 36894, done.
remote: Counting objects: 100% (36894/36894), done.
remote: Compressing objects: 100% (17843/17843), done.
remote: Total 36894 (delta 191), reused 36697 (delta 191), pack-reused 0 (from 0)
Receiving objects: 100% (36894/36894), 245.89 MiB | 18.26 MiB/s, done.
Resolving deltas: 100% (191/191), done.
Updating files: 100% (17442/17442), done.
17436


aozorahack/aozorabunko_text から『吾輩は猫である』のテキストデータを抽出して前処理を行う。

In [12]:
#@title 青空文庫のテキストデータをダウンロードする（neko1.txt）
!curl -RLO https://github.com/aozorabunko/aozorabunko/raw/refs/heads/master/cards/000148/files/789_ruby_5639.zip
!unzip -o 789_ruby_5639.zip wagahaiwa_nekodearu.txt
!mv wagahaiwa_nekodearu.txt neko1.txt
!rm -f 789_ruby_5639.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  336k  100  336k    0     0   701k      0 --:--:-- --:--:-- --:--:--  701k
Archive:  789_ruby_5639.zip
Made with MacWinZipper (http://tidajapan.com/macwinzipper)
  inflating: wagahaiwa_nekodearu.txt  


In [13]:
import re

def aozora_preprocess(text):
  # 冒頭の記載事項（タイトル・著者名等）を検出して除去する
  try:
    pos = text.index('\n\n')
    text = text[pos+2:]
  except:
    pass

  # 【テキスト中に現れる記号について】を検出して除去する
  try:
    # 最初の行が「--------」で始まることを確認する
    assert text.startswith('--------')

    # ２つ目の「--------」で始まる行と，その次の空行を除去する
    pos = text.index('\n') + 1
    pos = text.index('--------', pos)
    pos = text.index('\n\n', pos) + 2
    text = text[pos:]
  except:
    pass

  # 本文の末尾の情報（底本，入力・校正にかんする情報等）を検出して除去する
  try:
    pos = text.index('\n\n\n\n')
    text = text[:pos]
  except:
    pass

  # ルビに関する記述を削除する
  text = re.sub('《.+?》', '', text)
  text = re.sub('｜', '', text)

  # 注記情報（レイアウト情報，入力者注等）を検出して削除する
  text = re.sub(r'［＃.*?］', '', text)

  # 文ごとに改行する
  # text = re.sub('(。|？)', '\1\n', text)

  # HORIZONTAL BAR (U+2015) を EM DASH (U+2014) に変換する
  # text = text.replace('\u2015', '\u2014')

  return text

In [14]:
# テキストデータを読み込む
text1 = open('neko1.txt', encoding='cp932').read()

# 前処理を適用する
text1 = aozora_preprocess(text1)

In [ ]:
#@title Hugging Face 青空文庫のテキストデータをダウンロードする（neko2.txt）
ds = load_dataset('globis-university/aozorabunko-clean')
temp = ds.filter(lambda row: row['meta']['作品ID'] == '000789')
assert temp['train'].num_rows == 1
book = temp['train'][0]
text2 = book['text']

In [16]:
%%time
#@title ２つのテキストデータを比較する

from difflib import SequenceMatcher

matcher = SequenceMatcher(None, text1, text2)

for tag, i1, i2, j1, j2 in matcher.get_opcodes():
  if tag != 'equal':
    print(f"{text1[i1:i2]}\t{text2[j1:j2]}")

※	譃
※	饀
※	蹰
※	燄
※	饍
※	炷
※	炷
※	炷
※	炷
※	ㇶ
※	ㇶ
※	橛
※	睜
※	蹰
※※	惝怳
※	媧
※	鄢
※	燄
※	戕
※	匇
※	餼
※	騃
※	噱
※	彘
※	惸
※	璆
※	泫
※	蛼
※	蛼
※	燄
※	ㇶ
※	卺
※	燄
CPU times: user 1min 19s, sys: 83.2 ms, total: 1min 19s
Wall time: 1min 25s


## 自前の前処理済みテキストとも比較する

In [17]:
#@title 自前のテキストデータをダウンロードする（neko3.txt）
!curl -RLO https://github.com/dfukagaw28/aozorabunko_text/raw/refs/heads/preprocess/cards/000148/files/789_ruby_5639/789_ruby_5639.txt
!mv 789_ruby_5639.txt neko3.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1088k  100 1088k    0     0  1346k      0 --:--:-- --:--:-- --:--:-- 1346k


In [18]:
# 追加の前処理

def _preprocess(text):

  # ルビに関する記述を削除する
  text = re.sub('《.+?》', '', text)
  text = re.sub('｜', '', text)

  # 注記情報（レイアウト情報，入力者注等）を検出して削除する
  text = re.sub(r'［＃.*?］', '', text)

  # 前後の空白を除去する
  text = text.strip()

  return text

In [19]:
# テキストデータを読み込む
text3 = open('neko3.txt', encoding='utf8').read()

# 前処理を適用する
text3 = _preprocess(text3)

In [20]:
# ２つのテキストは一致する
assert text2 == text3